# Cluster Spotify Songs

The data consist of 25 parameters from ~300k spotify songs.
These parameters include identifying information (e.g., artist, album, track_id) as well as various song-related features (e.g., 'danceability', 'energy', 'key').
We're going to explore different methods for clustering these songs.
Accurate clustering would allow a recommendation algorithm to suggest new songs



#### load modules

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

#### load data

In [ ]:
data_path = '/Users/andrew/Documents/GitHub/data_science/song_clustering/data/'
df = pd.read_csv(data_path + 'tracks_features_subsample.csv')

#### Description of categories
(descriptions taken from Spotify's website)
Starting with the less-than-obvious columns.

danceability: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

energy: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

acousticness: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.

instrumentalness: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.

liveness: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.

loudness: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.

mode: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.

speechiness: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

valence: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

key: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.

time_signature: An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4".

tempo: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

## Group tracks into genres

In [ ]:
# start by pulling out all of the relevant columns
X_all = df[['explicit','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','valence','tempo','duration_ms','time_signature','year']]

# further downselect to only the non-categorical features
X_cont = X_all[['danceability','energy','loudness','speechiness','acousticness','instrumentalness','valence','tempo','duration_ms','year']]

# normalize loudness, tempo, duration_ms, and year to be between 0-1 (like the other variables)
temp_list = ['tempo','duration_ms','year','loudness']
for ii in temp_list:
    d = X_cont[ii]
    X_cont.loc[:,[ii]] = (d - np.min(d)) / (np.max(d) - np.min(d))

In [ ]:
# run PCA on the continuous variables
from sklearn.decomposition import PCA

# convert to array
X_cont_array = np.array(X_cont)

# fit model
pca = PCA(n_components = None)
proj = pca.fit_transform(X_cont_array);

# pull out weights
W = pca.components_

# pull out the explained variance (fraction of total variance)
varExp = pca.explained_variance_ratio_

# plot variance explained
plt.figure(figsize=(10,5))
plt.subplot(121);
plt.plot(varExp);
plt.title('fractional variance explained by PCs');

# plot the loadings for the PCs
plt.subplot(122);
plt.imshow(W, cmap='PuOr');
plt.colorbar();
plt.ylabel('PCs');
plt.clim([-0.75, 0.75]);
plt.xticks(np.arange(proj.shape[1]), labels = X_cont.columns, rotation=90);
plt.title('loadings for PCs');



### duration and year aren't doing us any good (PCs 9 and 10 are just duration and year), so toss these and re-run PCA

In [ ]:
# drop duration and year
X_cont.drop(['duration_ms','year'],axis=1, inplace=True)

# convert to array
X_cont_array = np.array(X_cont)

# fit model
pca = PCA(n_components = None)
proj = pca.fit_transform(X_cont_array);

# pull out weights
W = pca.components_

# pull out the explained variance (fraction of total variance)
varExp = pca.explained_variance_ratio_

# plot variance explained
plt.figure(figsize=(10,5))
plt.subplot(121);
plt.plot(varExp);
plt.title('fractional variance explained by PCs');

# plot the loadings for the PCs
plt.subplot(122);
plt.imshow(W, cmap='PuOr');
plt.colorbar();
plt.ylabel('PCs');
plt.clim([-0.75, 0.75]);
plt.xticks(np.arange(proj.shape[1]), labels = X_cont.columns, rotation=90);
plt.title('loadings for PCs');

In [ ]:
# plot the songs in a few different spaces
plt.figure(figsize=(12,3));
spaces_to_plot = np.arange(8).reshape(-1,2,order='C');
for ii in range(4):
    ax = plt.subplot(1,4,ii+1);
    plt.plot(proj[:,spaces_to_plot[ii,0]],proj[:,spaces_to_plot[ii,1]], '.', ms=0.01);
    plt.xlabel('PC ' + str(spaces_to_plot[ii,0]))
    plt.ylabel('PC ' + str(spaces_to_plot[ii,1]))
    ax.set_xticks([]);
    ax.set_yticks([]);

#### do a bit of preprocessing
convert categorical variables to dummy variables.
not normalizing or centering because non-categorical variables are already on about the same scale

In [ ]:
# Convert categorical (or binary) variables to dummy variables (one-hot encoded)
catVars = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'MTRANS']

for ii in catVars:

    # create the dummy variable
    dummy = pd.get_dummies(df[ii], drop_first=True)

    # remove the original variable
    df.drop(ii, axis = 1, inplace=True)

    # add the dummified variable
    df = pd.concat([df, dummy],axis=1)

# drop height and weight from table
df.drop(['Height','Weight'], axis=1, inplace=True)

# convert the variable referencing 'consumption of food between meals' to integer values
vals = np.arange(4)
keywords = ['no', 'Sometimes', 'Frequently', 'Always']
for ii, kw in enumerate(keywords):
    df.loc[df['CAEC']==kw, 'CAEC'] = vals[ii]

# do the same with alcohol consumption
vals = np.arange(3)
keywords = ['no','Sometimes','Frequently']
for ii, kw in enumerate(keywords):
    df.loc[df['CALC']==kw, 'CALC'] = vals[ii]

# and the obesity level
vals = np.arange(7)
keywords = ['Insufficient_Weight','Normal_Weight','Overweight_Level_I','Overweight_Level_II',
            'Obesity_Type_I','Obesity_Type_II','Obesity_Type_III']
for ii, kw in enumerate(keywords):
    df.loc[df['NObeyesdad']==kw, 'NObeyesdad'] = vals[ii]

# convert the obesity level to a binary variable
df.loc[df['NObeyesdad'] < 4, 'NObeyesdad'] = 0
df.loc[df['NObeyesdad'] >= 4,'NObeyesdad'] = 1

# toss the id column
df.drop('id', axis=1, inplace=True)

In [ ]:
# separate out our training and test data
# fraction of data to use for training
train_fraction = 0.9
num_train = round(df.shape[0] * train_fraction)

df_train = df.iloc[:num_train]
df_test  = df.iloc[num_train:]

#### Logistic regression
This will be our baseline
no regularization

In [ ]:
from sklearn.linear_model import LogisticRegression

# pull out the train X and train Y data
X_train = np.array(df_train.iloc[:, df.columns != 'NObeyesdad'])
Y_train = np.array(df_train.iloc[:, df.columns == 'NObeyesdad']).astype('bool')[:,0]

# pull out the test data
X_test = np.array(df_test.iloc[:, df.columns != 'NObeyesdad'])
Y_test = np.array(df_test.iloc[:, df.columns == 'NObeyesdad']).astype('bool')[:,0]

# initialize our model
model_logReg = LogisticRegression(penalty=None, max_iter=500)

# fit the model
model_logReg.fit(X_train,Y_train)

# test the model
Y_hat = model_logReg.predict(X_test)

# calculate accuracy
baseline_score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(baseline_score))

# initialize a dictionary to keep track of our scores
all_scores = {'baseline': baseline_score}

#### Not bad (certainly above chance), but we can clearly do better
as a simple start, at L2 regularization to combat overfitting (likely not an issue, given the small number of parameters)

In [ ]:
# magnitude of L2 penalty (0-1, with lower values corresponding to a greater penalty)
l2_lam = np.logspace(-5,stop=1, num=8)[::-1]
num_lams = len(l2_lam)

# fit each in turn
model_logReg_l2 = [None] * num_lams

# list to keep track of scores with different l2 penalties
l2_score = []
for ii,ll in enumerate(l2_lam):

    model_logReg_l2[ii] = LogisticRegression(penalty='l2', C=ll, max_iter=500 )

    # fit the model
    model_logReg_l2[ii].fit(X_train,Y_train)

    # test the model
    Y_hat = model_logReg_l2[ii].predict(X_test)

    # calculate accuracy
    score = sum(Y_test == Y_hat) / len(Y_test)
    l2_score.append(score)

    # display result as fractional improvement over baseline
    pct_dScore = (score - baseline_score) / baseline_score
    print('percent change in classification accuracy: ' + str(pct_dScore))

# add the best score to the dictionary
all_scores['l2_reg'] = np.max(l2_score)

#### Very tiny improvement in performance with moderate normalization

#### Before we change up the model architecture, let's see if we can get any improvements by handling the input data more intelligently (i.e., 'feature engineering') .
First, we're going to bin the ages by decade (whether a person is 30 or 33 likely does not greatly change their likelihood of being obese)
We're also going to log-transform the ages, as the fraction of obese individuals changes dramatically as a function of age during early-middle age


In [ ]:
# add a 'binned age' column to the dataframe
df['age_bin'] = pd.cut(df['Age'], bins=[10,20,30,40,50,60,70],labels=[0,1,2,3,4,5])

# log transform the ages
df['log_age'] = np.log1p(df['Age'])

In [ ]:
# pull out the test and train data again
X_df = df.copy()
Y = np.array(X_df['NObeyesdad']).astype('bool')
X_df.drop(['NObeyesdad'],axis=1,inplace=True)
X = np.array(X_df)


# pull out the train X and train Y data
X_train = X[:num_train,:]
Y_train = Y[:num_train]

# pull out the test data
X_test = X[num_train:,:]
Y_test = Y[num_train:]

In [ ]:
# train models (with l2 regularization and feature engineering)
model_logReg_l2_fe = [None] * num_lams

# list to keep track of scores
fe_scores = []
for ii,ll in enumerate(l2_lam):

    model_logReg_l2_fe[ii] = LogisticRegression(penalty='l2', C=ll, max_iter = 500 )

    # fit the model
    model_logReg_l2_fe[ii].fit(X_train,Y_train)

    # test the model
    Y_hat = model_logReg_l2_fe[ii].predict(X_test)

    # calculate accuracy
    score = sum(Y_test == Y_hat) / len(Y_test)
    fe_scores.append(score)

    # display result as fractional improvement over baseline
    pct_dScore = (score - baseline_score) / baseline_score
    print('percent change in classification accuracy: ' + str(pct_dScore))

# save the best score
all_scores['feature_engineering'] = np.max(fe_scores)

#### So that feature engineering (plus regularization) brought us from ~0.8 to ~0.82 accuracy

#### Now it's time to switch things up with a different model architecture
We're going to start with a feedforward network (relu units, trained with dropout)

In [ ]:
# define some parameters for our network
num_units_per_layer = 300
num_hidden_layers = 10
batch_size = 1000
num_epochs = 500
dropout_p  = 0.1
input_dim = X_train.shape[1]

In [ ]:
import sys
sys.path.append('/Users/andrew/Documents/GitHub/data_science/obesity_prediction/code/')
from torch.utils.data import DataLoader
import utils
import torch

# define the dataset and dataloader (both make passing data to the model easier)
dataset    = utils.FF_dataset(data=X_train.astype('float16'), targets=Y_train.astype('float16').reshape(-1,1))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# initialize the network
net = utils.FF_network(input_dim=input_dim, num_hidden_layers=num_hidden_layers,
                               num_units_per_layer=num_units_per_layer, dropout_p=dropout_p)
# define an optimizer (using default ADAM parameters)
# get a list of the named parameters
optimizer = torch.optim.Adam(net.parameters())

# use a simple MSE loss function
criterion = torch.nn.BCELoss()

# train the network
allLoss = []

for epoch in range(num_epochs):

    for ii, (x, y) in enumerate(dataloader):

        # make sure we're in training mode (the drop-out layers are active)
        net.train()

        # clear the gradient
        optimizer.zero_grad()

        # forward pass
        out = net(x)

        # calculate loss
        loss = criterion(out, y)
        allLoss.append(loss.detach().numpy())

        # calculate gradient
        loss.backward()

        # update parameters
        optimizer.step()

# plot the loss across batches
plt.plot(allLoss);

In [ ]:
# test on the held-out data
net.eval()
Y_hat = net(torch.Tensor(X_test.astype('float16')))
Y_hat = Y_hat.detach().numpy()[:,0]

# convert to prediction
Y_hat[Y_hat<0.5] = 0
Y_hat[Y_hat>=0.5] = 1

# calculate accuracy
score = sum(Y_test == Y_hat) / len(Y_test)

# display result as fractional improvement over baseline
pct_dScore = (score - baseline_score) / baseline_score
print('classification accuracy: ' + str(score))
print('percent change in classification accuracy: ' + str(pct_dScore))

# save the score
all_scores['ff_net'] = score

#### Now things are looking better.
Let's optimize the hyperparameters with a simple grid search
Will choose hyperparameters using performance on a validation set

In [ ]:
# define our hyperparameter grid
units_per_layer_grid = np.linspace(200,500,3)
num_layers_grid      = np.linspace(5,50,3)
p_dropout_grid       = np.linspace(0.05,0.6,3)

In [ ]:
# define our train and validation sets
percent_validation = 0.1
numValidation = round(X_train.shape[0] * percent_validation)

X_val = X_train[:numValidation,:]
Y_val = Y_train[:numValidation]
X_train_search = X_train[numValidation:,:]
Y_train_search = Y_train[numValidation:]

# each model is going to be trained until the validation error starts to increase.
# define the number of epochs in each 'cycle'
num_epochs_per_cycle = 15

# define a new dataset and dataloader
dataset_search    = utils.FF_dataset(data=X_train_search.astype('float16'), targets=Y_train_search.astype('float16').reshape(-1,1))
dataloader_search = DataLoader(dataset_search, batch_size=batch_size, shuffle=True)

In [ ]:
# initialize array to hold validation set performance
val_accuracy = np.zeros((3,3,3)) * np.nan

# cycle through the hyperparameters
for ii,units in enumerate(units_per_layer_grid):
    for jj, layers in enumerate(num_layers_grid):
        for kk, pDrop in enumerate(p_dropout_grid):

            # initialize the network
            net = utils.FF_network(input_dim=input_dim, num_hidden_layers=int(layers),
                                           num_units_per_layer=int(units), dropout_p=pDrop)

            # define an optimizer (using default ADAM parameters)
            # get a list of the named parameters
            optimizer = torch.optim.Adam(net.parameters())

            # use a simple MSE loss function
            criterion = torch.nn.BCELoss()

            # keep track of the validation loss
            valScore = -np.inf
            contFlg = True
            cycleCounter=0

            # run through epochs as long as the validation loss is decreasing
            while contFlg:
                display('cycle number: ' + str(cycleCounter))

                for epoch in range(num_epochs_per_cycle):

                    for i, (x, y) in enumerate(dataloader_search):

                        # make sure we're in training mode (the drop-out layers are active)
                        net.train()

                        # clear the gradient
                        optimizer.zero_grad()

                        # forward pass
                        out = net(x)

                        # calculate loss
                        loss = criterion(out, y)

                        # calculate gradient
                        loss.backward()

                        # update parameters
                        optimizer.step()

                # calculate loss on validation data
                net.eval()
                Y_hat = net(torch.Tensor(X_val.astype('float16')))
                Y_hat = Y_hat.detach().numpy()[:,0]

                # convert to prediction
                Y_hat[Y_hat<0.5] = 0
                Y_hat[Y_hat>=0.5] = 1

                # calculate accuracy
                score = sum(Y_val == Y_hat) / len(Y_test)

                # check to see if validation loss decreased
                if score > valScore:
                    valScore = np.copy(score)
                    cycleCounter+=1
                else:
                    contFlg = False
                    val_accuracy[ii,jj,kk] = valScore


In [ ]:
import scipy

# interpolate to get higher resolution
units_per_layer_grid_int = np.linspace(200,500,10)
num_layers_grid_int     = np.linspace(5,50,10)
p_dropout_grid_int      = np.linspace(0.05,0.6,10)

# generate 2D array of our known data points
xv, yv, zv = np.meshgrid(units_per_layer_grid, num_layers_grid, p_dropout_grid, indexing='ij')
points = np.concatenate([x.reshape(-1,1,order='F') for x in [xv, yv, zv]],axis=1)

# make a 2D array of the desired data point
xv, yv, zv = np.meshgrid(units_per_layer_grid_int, num_layers_grid_int, p_dropout_grid_int, indexing='ij')

# flatten our values
val_flat = val_accuracy.reshape(-1,1,order='F');

# interpolate
interp_grid = scipy.interpolate.griddata(points, val_flat, (xv, yv, zv), method='linear')

# find the maximum (interpolated) value)
m_ii, m_jj, m_kk = np.unravel_index(np.argmax(interp_grid), (10,10,10), order='C')

# pull out the parameters associated with the maximum value
units_per_layer_opt = units_per_layer_grid_int[m_ii]
num_layers_grid_opt = num_layers_grid_int[m_jj]
p_dropout_grid_opt  = p_dropout_grid_int[m_kk]

In [ ]:
# train a new network with these parameters

# up the number of epochs
num_epochs = 1000

# initialize the network
net = utils.FF_network(input_dim=input_dim, num_hidden_layers=int(num_layers_grid_opt),
                               num_units_per_layer=int(units_per_layer_opt), dropout_p=p_dropout_grid_opt)

# define an optimizer (using default ADAM parameters)
# get a list of the named parameters
optimizer = torch.optim.Adam(net.parameters())

# use a simple MSE loss function
criterion = torch.nn.BCELoss()

# train the network
allLoss = []

for epoch in range(num_epochs):

    for ii, (x, y) in enumerate(dataloader):

        # make sure we're in training mode (the drop-out layers are active)
        net.train()

        # clear the gradient
        optimizer.zero_grad()

        # forward pass
        out = net(x)

        # calculate loss
        loss = criterion(out, y)
        allLoss.append(loss.detach().numpy())

        # calculate gradient
        loss.backward()

        # update parameters
        optimizer.step()

# plot the loss
plt.plot(allLoss);

In [ ]:
# test on the held-out data
net.eval()
Y_hat = net(torch.Tensor(X_test.astype('float16')))
Y_hat = Y_hat.detach().numpy()[:,0]

# convert to prediction
Y_hat[Y_hat<0.5] = 0
Y_hat[Y_hat>=0.5] = 1

# calculate accuracy
score = sum(Y_test == Y_hat) / len(Y_test)

# save accuracy
all_scores['ff_net_opt'] = score

# display result as fractional improvement over baseline
pct_dScore = (score - baseline_score) / baseline_score
print('classification accuracy: ' + str(score))
print('percent change in classification accuracy: ' + str(pct_dScore))

#### Things are looking much better now, but let's see how much things improve by 'ensembling' models

### our first ensemble is going to be a random forest classifier.
Before we get to that, let's look at how a single tree performs

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,Y_train)
Y_hat = clf.predict(X_test)

# calculate accuracy
score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(score))

# save score
all_scores['decision_tree'] = score

#### surprisingly, a simple decision tree nearly beats our optimized network!

#### train a random forest classifier
very simple emsemble method. We're going to train a bunch of individual decision trees on bootstrapped collections of the data. This method nicely combats overfitting.
the final prediction is made by allowing each tree to 'vote'. Most popular classification wins.


In [ ]:
from numpy.random import choice

# number of trees in our forest
num_trees = 100

# for each tree, bootstrap our training data

# number of training samples
num_samples  = X_train.shape[0]

# list to hold our trees
Trees = []
for ii in range(num_trees):

    # get the bootstrapped data
    idx = choice(np.arange(num_samples), num_samples)
    Trees.append(tree.DecisionTreeClassifier().fit(X_train[idx,:], Y_train[idx]))



In [ ]:
# get the predictions of each tree within the forest (on the test data). Count votes to decide final answer
test_out = [tree.predict(X_test) for tree in Trees]

# convert into array
test_out_array = np.array(test_out)

# find most popular output for each input
Y_hat = np.sum(test_out_array, axis=0) >= (num_trees/2)

# calculate accuracy
score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(score))

# save result
all_scores['random_forest_scratch'] = score

#### the random forest classifier is beating our network handily!
just for comparison, use the built-in random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
clf.fit(X_train,Y_train)

Y_hat = clf.predict(X_test)

# calculate accuracy
score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(score))

# save result
all_scores['random_forest_sklearn'] = score

#### Now we're easily beating our network
Seems like we top out at ~0.907 classification accuracy

#### Try a gradient boosting classifier
Another popular ensemble method. Here, we add decision trees linearly, and train each successive tree on the gradient (loss wrt change in tree parameters) of the previous tree.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(validation_fraction = 0.1, n_estimators = 1000)
clf.fit(X_train,Y_train)

Y_hat = clf.predict(X_test)

# calculate accuracy
score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(score))

# save result
all_scores['gradiant_boosting'] = score

#### The kitchen sink
Here, we're going to train a few different ensemble methods, and use the output of each ensemble as input to a final logistic classifier.
While this method could very likely outperform all others (for obvious reasons), doing so may require a significant amount of hyperparameter optimization.

In [ ]:
# import all of the classifiers we haven't already
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

# number of estimators per base ensemble
n_estimators = 500

# make a list of the estimators
estimators = [('ada', AdaBoostClassifier(n_estimators = n_estimators)),
              ('bag', BaggingClassifier(n_estimators = n_estimators)),
              ('ext', ExtraTreesClassifier(n_estimators = n_estimators)),
              ('grd', GradientBoostingClassifier(n_estimators = n_estimators)),
              ('rdm', RandomForestClassifier(n_estimators = n_estimators))]

clf = StackingClassifier(estimators = estimators, final_estimator=LogisticRegression(), n_jobs=-1)
clf.fit(X_train,Y_train)

In [ ]:
# check accuracy
Y_hat = clf.predict(X_test)

# calculate accuracy
score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(score))

# save score
all_scores['kitchen_sink_emsemble'] = score

#### Summarize performance of all of our classifiers

In [ ]:
plt.figure(figsize=(25,10));

# number of classifiers
num_class = len(all_scores)

# performance of each classifier
score_vals = list(all_scores.values())

# name of each classifier
class_names = all_scores.keys()

# plot
for ii in range(num_class):

    plt.bar(ii,score_vals[ii]);

# clean up
plt.xlim([-1,num_class]);
plt.ylim([0.5, 1]);
plt.ylabel('R$^2$');
plt.xlabel('classifier');
plt.xticks(ticks=np.arange(num_class), labels=class_names);


#### Summary of training various classifiers

decision tree (and ensembles of decision trees) perform comparably (or better) than feedforward networks, while requiring a fraction of the training time.

I only got a very modest improvement with my hyperparameter grid search (for the FF network). I likely would see a larger improvement with a more comprehensive/more intelligent search. For example, rather than a simple grid search, I could've used bayesian optimization.

## Explore the dataset
now that we've maxed out our classifier performance, let's use a few of the simpler classifiers to explore the relationship between the various variables and obesity

In [ ]:
# pull out the names of each of the variables
# we appended the dummy variables in the order: 'Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'MTRANS'

# pull out the variable names, drop 'NObeyesdad', replace the dummy variable names with more informative names
var_names = list(df.columns[df.columns != 'NObeyesdad'])

# find the index where we started appending dummy variables
gIdx = [ii for ii, x in enumerate(var_names) if x=='Male' or x=='Female'][0]

# replace with more informative names
var_names[gIdx] = 'Gender'
var_names[gIdx+1] = 'family_history_overweight'
var_names[gIdx+2] = 'FAVC'
var_names[gIdx+3] = 'smoke'
var_names[gIdx+4] = 'SCC'
transport = ['bike','motorbike','public_transit','walking']
for ii in range(5,9):
    var_names[gIdx+ii] = transport[ii-5]


In [ ]:
# train a logistic regression classifier
model = LogisticRegression(penalty='l2', C=1e-2, max_iter=500 )

# fit the model
model.fit(X_train,Y_train)

# test the model
Y_hat = model.predict(X_test)

# calculate and display accuracy
score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(score))

# pull out the coefficients
coefs = model.coef_[0,:]

# plot the coefficients
plt.figure(figsize=(30,5));
plt.plot(abs(coefs));
plt.xticks(np.arange(len(coefs)), labels=var_names);

we didn't normalize our variables, so we can only get a rough idea about how 'important' each of the variables are to the prediction. But from a first pass, it seems that family history of being overweight, consuming food between meals, and whether or not the person takes public transportation to work are the most informative variables

To verify this impression, get p-values for each of the coefficients

In [ ]:
result.pvalues

In [ ]:
import statsmodels.api as sm

model=sm.Logit(Y_train.astype('float32'),X_train.astype('float32'))
result=model.fit()

# plot the inverse of the P-values
# sort the p-values
idx = np.argsort(result.pvalues)

# print the variables in order
sorted_vars = [var_names[x] for x in idx]
print('sorted variables: ')
print(sorted_vars);


This is why we need to be careful about interpreting the magnitude of the coefficients if we don't normalize them! 'Log-age' is by far the 'most important' variable (p < 1e-263, while the next closest variable has a value of < 1e-200).

In [ ]:
# I'm now going to re-train the model with only the 5 most 'useful' variables
X_train_ds = X_train[:,idx[:5]]

# train a logistic regression classifier
model = LogisticRegression(penalty='l2', C=1e-2, max_iter=500 )

# fit the model
model.fit(X_train_ds,Y_train)

# test the model
Y_hat = model.predict(X_test[:,idx[:5]])

# calculate and display accuracy
score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(score))

#

after getting rid of 14 of the 19 variables, we're still able to predict nearly as well as if we used all of the variables (98% as well).
In fact, if we only use 'log age', we can predict 89% as well (0.72 prediction accuracy on held-out data).

In [ ]:
# I'm now going to re-train the model with only the 5 most 'useful' variables
X_train_ds = X_train[:,idx[:1]]

# train a logistic regression classifier
model = LogisticRegression(penalty='l2', C=1e-2, max_iter=500 )

# fit the model
model.fit(X_train_ds,Y_train)

# test the model
Y_hat = model.predict(X_test[:,idx[:1]])

# calculate and display accuracy
score = sum(Y_test == Y_hat) / len(Y_test)
print('classification accuracy: ' + str(score))

### To verify that this is true, do a simple analysis where we define single threshold. If an individual's log age is greater than that threshold, they're predicted to be obese.
Sweep this threshold across a range and ask how our performance changes.

In [ ]:
# define our thresholds
threshold = np.linspace(2.8,3.8,num=20);

# pull out the
log_age = X_test[:,idx[0]]

# keep track of our scores
scores = []

# run analysis
for tt in threshold:
    Y_hat = np.zeros_like(Y_test)
    Y_hat[log_age >= tt] = 1

    # calculate score
    score = sum(Y_test == Y_hat) / len(Y_test)
    scores.append(score)

# plot results
plt.plot(threshold,scores);
plt.xlabel('log age threshold');
plt.ylabel('R$^2$');
plt.title('predicting obesity using only log age');